In [46]:
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import matplotlib.pyplot as plt

#Función para calcular el tiempo con un modelo exponencial

def calcula_tiempo(tasa):
    random_number = np.random.rand(1)
    tiempo = (-1/tasa)*math.log(random_number) 


    return tiempo
    

In [63]:
#Función para crear un DataFrame a partir de los archivos entregados para las rutas de cada red
#Los archivos se encuentran en una subcarpeta de nombre 'rutas'
N = 0
L = 0

def create_df(file_name):
    df = pd.read_csv('./rutas/'+file_name,header=None)
    df = df[0].str.split('\t', expand=True)
    N = int(df[3][1])
    L = int(df[3][2]) - 1
    return df, N, L


def clean_df_and_C(df):
    df = df[df[0] != df[1]][6:].reset_index(drop=True)
    C = df.iloc[:,3:].apply(pd.value_counts).sum(axis=1)
    df['rutas'] = df.iloc[:,3:].apply(
            lambda x: [i for i in x.dropna().astype(int)],
            axis=1
        )
    df['indice'] = C
    return df, C

df, N, L = create_df('EuroCore-WDM.rut')
df, C = clean_df_and_C(df)

#Usuarios
M = N*(N-1)

#display(df)
print('Usuarios :',M,' Enlaces',L)



def calculo_longitudes_enlaces(C,porc_dimensionamiento):

    indices = C.index.tolist()

    C = C.tolist()
    indices = list(map(int,indices))

    long = []

    for x in range(L+1):
        long.append(0)
    count = 0
    for i in indices:
        long[i] = math.ceil(C[count]*porc_dimensionamiento)
        count += 1
    #print(long)

    return long

long = calculo_longitudes_enlaces(C,0.6)

Usuarios : 110  Enlaces 49


In [60]:
def create_FEL(enlaces,df,M):
    t_off = 20
    FEL = []
    i = 0
    while (i < M):
    
        FEL.append(['T',i,'OFF',calcula_tiempo(t_off),df['rutas'][i],df['rutas'][i+1]])
        i += 2
    
    for x in range(enlaces):
        FEL.append(['F',x,'OK',calcula_tiempo(t_off)])

    FEL.sort(key=itemgetter(3))

    return FEL

FEL = create_FEL(L,df,M)
#FEL



In [65]:
def simulator(FEL,capacidad):
    #Tasa de transmision, de falla 
    t_off = 20
    t_on = 1
    t_falla = 20
    #---------------------------- 
      
    #transmisiones a simmular
    TRANSMISIONES = 300
    #----------------------------

    #metricas a calcular
    fallo_ruta_primaria = 0
    fallo_ruta_secundaria = 0
    bloqueos = 0
    bloqueo_capacidad = 0
    fallas_ocurridas = 0
    transmisiones = 0
    enlace_fallido = 'ninguno'
    #----------------------------

    FEL_sim = FEL
    C_sim = np.array(capacidad)
    falla = []


    while (transmisiones <= TRANSMISIONES):
        
        evento_actual = FEL_sim.pop(0)                  #obtenemos el evento que viene en el tiempo siguiente

        if (evento_actual[0] == 'F'):                   #si es una falla entra aqui

            falla = evento_actual                
            fallas_ocurridas += 1
                                                 
            falla[3] = falla[3] + calcula_tiempo(t_falla)   #sumamos tiempo para que esta falla vuelva a ocurrir mas adelante
                                                
            enlace_fallido = falla[1]
            
            FEL_sim.append(falla)                   #agregamos la falla de nuevo a la FEL de eventos con un tiempo mayor
            #print('Enlace con falla: ',enlace_fallido)
            
        if (evento_actual[0] == 'T'):               #si es una transmision entra aqui
            
            if (evento_actual[2] == 'OFF'):         #si esta en OFF, es porque va a transmitir

                transmisiones += 1
                ruta_primaria = evento_actual[4]        #obtenemos la ruta primaria que usara la transmision
                ruta_secundaria = evento_actual[5]      #obtenemos la ruta secundaria que usara la transmision

                #print('Ruta primaria: ',ruta_primaria)
                #print('Ruta secundaria: ',ruta_secundaria)

                if (enlace_fallido in ruta_primaria):   #si el enlace que esta con falla esta presente en la ruta primaria entra aqui
                    fallo_ruta_primaria += 1

                    #comienza proceso de transmision por la ruta secundaria
                    
                    if (np.all(C_sim[ruta_secundaria] > 0) ):             #verificamos si hay long de onda disponibles en la red para transmitir
                        C_sim[ruta_secundaria] -= 1
                        #C_sim -= 1              #restamos 1 a la capacidad de transmision de la red, debido a que se tendran long de onda limitadas
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                        evento_actual[2] = 'ON'
                        FEL_sim.append(evento_actual)

                    else:
                        bloqueo_capacidad += 1
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                        FEL_sim.append(evento_actual)

                elif (enlace_fallido in ruta_secundaria):
                    fallo_ruta_secundaria += 1
                    #comienza proceso de transmision por la ruta secundaria
                    
                    if (np.all(C_sim[ruta_primaria] > 0) ):             #verificamos si hay long de onda disponibles en la red para transmitir
                        C_sim[ruta_primaria] -= 1
                        #C_sim -= 1              #restamos 1 a la capacidad de transmision de la red, debido a que se tendran long de onda limitadas
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                        evento_actual[2] = 'ON'
                        FEL_sim.append(evento_actual)

                    else:
                        bloqueo_capacidad += 1
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                        FEL_sim.append(evento_actual)

                
                #comienza proceso de transmision por la ruta primaria            
                else:
                    if (np.all(C_sim[ruta_primaria] > 0)):
                        C_sim[ruta_primaria] -= 1
                        #C_sim -= 2
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                        evento_actual[2] = 'ON'
                        FEL_sim.append(evento_actual)
                    
                    if (np.all(C_sim[ruta_secundaria] > 0)):
                        C_sim[ruta_secundaria] -= 1
                        if (np.all(C_sim[ruta_primaria] < 0)):
                            #C_sim -= 2
                            evento_actual[3] = evento_actual[3] + calcula_tiempo(t_on)
                            evento_actual[2] = 'ON'
                            FEL_sim.append(evento_actual) 

                    if(np.all(C_sim[ruta_primaria] < 0) and np.all(C_sim[ruta_secundaria] < 0)):
                        bloqueo_capacidad += 1
                        evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                        FEL_sim.append(evento_actual)

            elif (evento_actual[2] == 'ON'):        #si esta en ON, es porque terminará de transmitir
                C_sim[ruta_primaria] += 1
                C_sim[ruta_secundaria] += 1
                #C_sim += 1
                evento_actual[3] = evento_actual[3] + calcula_tiempo(t_off)
                evento_actual[2] = 'OFF'
                FEL_sim.append(evento_actual)

        #FEL_sim.append(evento_actual)
        FEL_sim.sort(key=itemgetter(3))

    print('Bloqueos: ',bloqueo_capacidad)
    print('Probabilidad de bloqueo con ',TRANSMISIONES,'transmisiones: ',bloqueo_capacidad / transmisiones )
    print('Fallo de rutas secundarias: ', fallo_ruta_primaria)
    print('Fallo de rutas secundarias: ', fallo_ruta_secundaria, '\n')
    return (bloqueo_capacidad/transmisiones)


simulator(FEL,long)

Bloqueos:  3
Probabilidad de bloqueo con  300 transmisiones:  0.009966777408637873
Fallo de rutas secundarias:  8
Fallo de rutas secundarias:  12 



'success'

In [72]:
def simulador_dimensionamiento(porcentaje):
    P = 0
    par_prob_dim = []
    while (porcentaje <= 1.0):
        print('Porcentaje de dimensionamient: ',porcentaje)
        long = calculo_longitudes_enlaces(C,porcentaje)
        FEL = create_FEL(L,df,M)
        P = simulator(FEL,long)
        par_prob_dim.append([porcentaje,P])
        porcentaje += 0.1

    return par_prob_dim
data = simulador_dimensionamiento(0.5)
        

Porcentaje de dimensionamient:  0.5
Bloqueos:  5
Probabilidad de bloqueo con  300 transmisiones:  0.016611295681063124
Fallo de rutas secundarias:  7
Fallo de rutas secundarias:  16 

Porcentaje de dimensionamient:  0.6
Bloqueos:  1
Probabilidad de bloqueo con  300 transmisiones:  0.0033222591362126247
Fallo de rutas secundarias:  8
Fallo de rutas secundarias:  18 

Porcentaje de dimensionamient:  0.7
Bloqueos:  0
Probabilidad de bloqueo con  300 transmisiones:  0.0
Fallo de rutas secundarias:  7
Fallo de rutas secundarias:  16 

Porcentaje de dimensionamient:  0.7999999999999999
Bloqueos:  4
Probabilidad de bloqueo con  300 transmisiones:  0.013289036544850499
Fallo de rutas secundarias:  5
Fallo de rutas secundarias:  17 

Porcentaje de dimensionamient:  0.8999999999999999
Bloqueos:  1
Probabilidad de bloqueo con  300 transmisiones:  0.0033222591362126247
Fallo de rutas secundarias:  4
Fallo de rutas secundarias:  9 

Porcentaje de dimensionamient:  0.9999999999999999
Bloqueos:  2
Pr